### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

**Question 1:** Set the multi-level index so the first level is the stock symbol itself, and the second level is the date.  Make sure the date column is sorted in ascending order.  You might have to use the `sort_index(level=0)` method to get the values straight.

**Important:** Make sure you use the `parse_dates` argument to read the date column as a date.  If you don't it might end up as a string.

In [1]:
# your answer here
import pandas as pd

# upload
stocks = pd.read_csv('../data/stocks_panel.csv', parse_dates=['Date'])

In [2]:
# set the index and sort everything to get things working correctly
stocks = stocks.set_index(['Stock', 'Date']).sort_index(level=0)

**Question 2:** To capture some other aspects of dates, create columns in your dataset that capture the following aspects of each timestamp:

  - What quarter it's in
  - Whether or not it's the last day of the month/quarter
  - What day it is (ie, do price changes vary by day?)
  
**Hint:** You don't use the `dt` attribute to get date parts from index values.  Multi indices are also a little tricky.  

To get what you want, try this: `df.index.get_level_values(level=1).your_datepart_here`

If you are looking for a refresher on different pandas date parts, you can find them at this url:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [3]:
# this gets days of week
stocks['dayofweek']  = stocks.index.get_level_values(level=1).dayofweek
# this gets the quarter
stocks['quarter']    = stocks.index.get_level_values(level=1).quarter
# and this gets whether or not it's the last day of the month
stocks['endofmonth'] = stocks.index.get_level_values(level=1).is_month_end
stocks['endofquarter'] = stocks.index.get_level_values(level=1).is_quarter_end

**Question 3:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

However, your data won't really "know" about those values unless they can be observed alongside the current observation.  Data is read in as rows, not columns.  

To that end, make two columns that capture the value of the following:

 - What the previous recorded price for each stock was
 - The stock price from two observations ago
 
**Remember:** This has to be done on a particular level of the index so make sure it's getting applied appropriately!

In [4]:
# this will get the value from one observation ago
stocks['1d'] = stocks.groupby(level=0)['Price'].shift()
# and this from two observations ago
stocks['2d'] = stocks.groupby(level=0)['Price'].shift(2)

**Question 4:** How did each stock price change compared to the S&P 500? 

Lots of times it's useful to see how something else moves with some other item that you're trying to track.  

In the data folder is a file called `s&p.csv`, and it contains the price history of the S&P 500 index for each day since its inception. See if you can upload it, and merge the `adj close` column into your dataset, so there's a column that displays the observed value of the index for every single price observation we have in our dataset.

**Hints:**
 - Merging on multi-level indices is tricky and prone to failure.  To make this a little bit easier, just use `reset_index()` to pop out the date column in the multi-index, and merge on it as if it were a regular column.
 - Make sure both date columns are actually encoded as dates, rather than strings, or else the merge won't work.

In [5]:
# your answer here
s_and_p = pd.read_csv('../data/s&p.csv', parse_dates=['Date'])

In [6]:
# this will do the merge
stocks = stocks.reset_index().merge(s_and_p[['Date', 'Adj Close']], on='Date')
# and this will get us back to the original multi index
stocks = stocks.set_index(['Stock', 'Date']).sort_index(level=0)

**Question 5:** Window Statistics

Lots of times, we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [7]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.head()

,Value
2020-01-01,-1.986030
2020-01-02,-0.535873
2020-01-03,-0.222935
2020-01-04,0.717649
2020-01-05,1.707738


In [8]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,-0.087801


You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 5 & 10 day moving averages for each stock inside the dataset.

**Note:** Do *not* try and merge them back into your dataset yet, just make sure you have the values showing up.

In [9]:
# this would be an example of the rolling 5 day average
stocks.groupby(level=0)['Price'].rolling(5).mean()

Stock  Stock  Date      
AAPL   AAPL   2014-11-05           NaN
              2014-11-06           NaN
              2014-11-07           NaN
              2014-11-10           NaN
              2014-11-11    109.020000
              2014-11-12    109.498000
              2014-11-13    110.322000
              2014-11-14    111.356000
              2014-11-17    112.387999
              2014-11-18    113.542000
              2014-11-19    114.225999
              2014-11-20    114.923999
              2014-11-21    115.381999
              2014-11-24    116.309999
              2014-11-25    116.735998
              2014-11-26    117.601999
              2014-11-28    118.125999
              2014-12-01    117.845999
              2014-12-02    117.045999
              2014-12-03    116.711999
              2014-12-04    116.009999
              2014-12-05    115.223999
              2014-12-08    114.689999
              2014-12-09    114.588001
              2014-12-10    113.792000


If you take a look at the index, you should notice that it has *three* levels to it, and not just two like before.  

Combining datasets with differing numbers of levels is cumbersome, and there's a decent amount of churn in what methods work from one version of Pandas to another.  

For now, try and get these values back into your original dataset by taking the following steps:

 - calculate the 5 & 10 day rolling averages for each stock price on the multilevel index, and save these as variables, and then use the `values` attribute for each one to drop the index and just get the column values (ask me about this if you have questions)
 - use `reset_index()` to unstack the index on your original dataframe
 - create new columns for the 5 & 10 day moving averages in the original dataset, using the values from the first step.
 
So as a quick example, it would sort of work like this:

`five_day = df.groupby(level=0)['Price'].your_stuff_here.values`

And then use this as the basis to make your new column from your original dataframe with the reset index.

In [10]:
# 5 & 10 day moving averages
five_day_vals = stocks.groupby(level=0)['Price'].rolling(5).mean().values
ten_day_vals  = stocks.groupby(level=0)['Price'].rolling(10).mean().values

In [11]:
# reset the index, and assign the rolling averages
stocks = stocks.reset_index()
stocks['5day']  = five_day_vals
stocks['10day'] = ten_day_vals

In [14]:
stocks.head()

,Stock,Date,Price,dayofweek,quarter,endofmonth,endofquarter,1d,2d,Adj Close,5day,10day
0,AAPL,2014-11-05,108.860001,2,4,False,False,NaN,NaN,2023.569946,NaN,NaN
1,AAPL,2014-11-06,108.699997,3,4,False,False,108.860001,NaN,2031.209961,NaN,NaN
2,AAPL,2014-11-07,109.010002,4,4,False,False,108.699997,108.860001,2031.920044,NaN,NaN
3,AAPL,2014-11-10,108.830002,0,4,False,False,109.010002,108.699997,2038.260010,NaN,NaN
4,AAPL,2014-11-11,109.699997,1,4,False,False,108.830002,109.010002,2039.680054,109.02,NaN
